In [15]:
import os
import cv2
import numpy as np

def davis_2x2_prepare_colorization_data_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        prepared_image = davis_2x2_process_quadrants(image)
        
        save_path = os.path.join(output_folder, filename)
        cv2.imwrite(save_path, prepared_image)

def davis_2x2_process_quadrants(image):
    h, w, c = image.shape

    # Extract individual quadrants
    top_left = image[:h//2, :w//2]      # GT for segmentation examples
    top_right = image[:h//2, w//2:]     # Segmented GT
    bottom_left = image[h//2:, :w//2]   # Test GT
    bottom_right = image[h//2:, w//2:]  # Segmented Test GT

    # Convert the top left (GT) to grayscale for our colorization example
    top_left_gray = cv2.cvtColor(top_left, cv2.COLOR_BGR2GRAY)
    top_left_colored = cv2.cvtColor(top_left_gray, cv2.COLOR_GRAY2BGR)

    # Convert the bottom left (Test GT) to grayscale
    bottom_left_gray = cv2.cvtColor(bottom_left, cv2.COLOR_BGR2GRAY)
    bottom_left_colored = cv2.cvtColor(bottom_left_gray, cv2.COLOR_GRAY2BGR)

    # Combine quadrants to form a new image for colorization tasks
    top_half = np.concatenate((top_left_colored, top_left), axis=1)  # Putting grayscale GT beside original GT
    bottom_half = np.concatenate((bottom_left_colored, bottom_left), axis=1)  # Putting grayscale Test GT beside original Test GT

    new_image = np.concatenate((top_half, bottom_half), axis=0)
    return new_image


def davis_2x2_prepare_colorization_data_videos(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for filename in os.listdir(input_folder):
        video_path = os.path.join(input_folder, filename)
        cap = cv2.VideoCapture(video_path)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        
        filename = "colorization_" + filename
        out_path = os.path.join(output_folder, filename)
        out = None

        while(cap.isOpened()):
            ret, frame = cap.read()
            if not ret:
                break
            
            prepared_frame = davis_2x2_process_quadrants(frame)
            
            if out is None:
                [h, w, d] = prepared_frame.shape
                out = cv2.VideoWriter(out_path, fourcc, 30.0, (w, h))
            
            out.write(prepared_frame)

        cap.release()
        if out is not None:
            out.release()


In [16]:
folder_A = "/shared/katop1234/video_inpainting/video_inpainting/test_images/single_davis_image_prompts"
folder_B = "/shared/katop1234/video_inpainting/video_inpainting/test_images/colorization_single_davis_image_prompts"
davis_2x2_prepare_colorization_data_images(folder_A, folder_B)